### This notebook extracts data from the annotated vcf, cleans up a little, and saves it as pickled dataframe

#### Import vcf and convert into dataframe. Extract features inside cells as necessary.

In [1]:
import csv
import gzip
import re
import pickle

import pandas as pd

In [2]:
# get columns names from tab delimited file
cv_columns = {}
with gzip.open("raw_vcf/clinvar.annotated.vcf.gz", "rt") as f:
    for metaline in f:
        if metaline.startswith("##INFO"):
            colname = re.search("ID=(\w+),", metaline.strip("#\n"))
            coldesc = re.search(".*Description=(.*)>", metaline.strip("#\n"))
            cv_columns[colname.group(1)] = coldesc.group(1).strip('"')

In [3]:
# read tab delimited
cv_df = pd.read_csv(
    "raw_vcf/clinvar.annotated.vcf.gz",
    sep="\t",
    skiprows=35,
    usecols=[0, 1, 2, 3, 4, 7], # rid of columns 5, 6
    header=None,
)

In [4]:
cv_df.rename(columns={0: "CHROM", 1: "POS", 2: "ID", 3: "REF", 4: "ALT"}, inplace=True)

In [5]:
# convert the long dictionary in column 7 to actual columns
def list_to_dict(l):
    """Convert list to dict."""
    return {k: v for k, v in (x.split("=") for x in l)}

cv_df = pd.concat(
    [
        cv_df.drop([7], axis=1),
        cv_df[7].str.split(";").apply(list_to_dict).apply(pd.Series),
    ],
    axis=1,
)

In [6]:
# the CSQ column has a bit different syntax. Unpack and convert into actual columns

# get column names for CSQ
with gzip.open("raw_vcf/clinvar.annotated.vcf.gz", "rt") as f:
        for line in f:
            if line.startswith("##INFO=<ID=CSQ"):
                m = re.search(r'.*Format: (.*)">', line)
                cols = m.group(1).split("|")

# pipe to dict
def CSQ_to_dict(l):
    '''
    Convert the pipe_separated values in the CSQ column to dict
    '''
    annotation_data = {}
    for csq_column, csq_value in zip(cols, l):
        annotation_data[csq_column] = csq_value
    return annotation_data
        
# convert and concat
cv_df = pd.concat(
    [
        cv_df.drop(['CSQ'], axis=1),
        cv_df['CSQ'].str.split("|").apply(CSQ_to_dict).apply(pd.Series),
    ],
    axis=1,
)

#### Narrow down rows to those only with clear targets. Catagorize target into three different bins. 

In [7]:
# narrow rows down to just unambiguous clinical diagnoses
unamb_stat = ['criteria_provided,_multiple_submitters,_no_conflicts','reviewed_by_expert_panel','practice_guideline ']
cv_df = cv_df.loc[cv_df['CLNREVSTAT'].isin(unamb_stat)]

In [8]:
# drop rows with just drug_response as a CLNSIG
cv_df = cv_df.drop(cv_df[cv_df['CLNSIG'] == 'drug_response'].index)

#convert ['CLNSIG'] into just three catagories
cv_df.loc[cv_df['CLNSIG'].str.startswith("Benign") | cv_df['CLNSIG'].str.startswith("Likely_benign"), ['CLNSIG']] = 'Benign_cat'
cv_df.loc[cv_df['CLNSIG'].str.startswith("Uncertain_significance"), ['CLNSIG']] = 'Uncertain_significance_cat'
cv_df.loc[cv_df['CLNSIG'].str.startswith("Pathogenic") | cv_df['CLNSIG'].str.startswith("Likely_pathogenic"), ['CLNSIG']] = 'Pathogenic_cat'

In [9]:
cv_df['CLNSIG'].value_counts()

Benign_cat                    28086
Uncertain_significance_cat    17223
Pathogenic_cat                12354
Name: CLNSIG, dtype: int64

#### Drop columns absolutely not needed, and save cleaned up file as pickled file

In [10]:
#drop columns not needed
cv_df = cv_df.drop(columns=['ALLELEID', 'CLNDISDB',
       'CLNHGVS', 'CLNREVSTAT', 'CLNVCSO', 'CLNVI',
       'GENEINFO', 'RS', 'CLNSIGCONF',
       'CLNDISDBINCL', 'CLNSIGINCL', 'SSR', 'DBVARID','Allele', 
       'IMPACT', 'SYMBOL', 'Gene','Feature_type','Feature','HGVSc','HGVSp','Existing_variation','DISTANCE',
       'STRAND','FLAGS','SYMBOL_SOURCE','HGNC_ID','TSL','APPRIS','REFSEQ_MATCH',
       'GIVEN_REF','USED_REF','MOTIF_NAME','MOTIF_POS','HIGH_INF_POS','MOTIF_SCORE_CHANGE','MPC'])

In [11]:
file = open('cv_df_extracted', 'wb')
pickle.dump(cv_df, file)
file.close()